# MongoDB via Command Line Shell (Bash)

# Outline 

- Establish Connection to Server
- Show Databases
- Import Data
- Show Collections
- Sample Documents
- Get Record Counts
- Get List of Distinct Fields
- Get Number of Distinct Values by Field
- Get List of Unique Values for a Field
- Get Count of Unique Values by Value for a Field
- CRUD Operations
	- Delete and Create
	- Read
	- Update
- Indexes
	- View Indexes
	- Create Indexes

# Establish Connection to Server

<p>The host and port must be specified for each command, unless you want it to default to localhost. Unfortunately, you cannot set them and have them apply to future commands. You can specify environment variables if you want to use \$HOST and \$PORT in future references instead of hard-coding.</p>

In [5]:
# I am working from a Jupyter notebook on Windows, 
# so the 'set' command would be appropriate

!set HOST=127.0.0.1
!set PORT=27017

In [ ]:
# Over Linux, I would use:

# !export HOST=127.0.0.1
# !export PORT=27017

To keep it simple, I will omit the host and port, since I am using localhost.

# Show Databases

In [2]:
# using --quiet
!mongosh --quiet --eval "show dbs"

admin        132.00 KiB
clickstream  456.26 MiB
config        96.00 KiB
local         96.00 KiB


In [32]:
# not using --quiet
!mongosh --eval "show dbs"

Current Mongosh Log ID:	683e47d4c5f8485f46e6e9f6
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.0
Using MongoDB:		7.0.4
Using Mongosh:		2.1.0
mongosh 2.5.1 is available for download: https://www.mongodb.com/try/download/shell

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2025-05-25T19:06:13.572-06:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
------

admin        132.00 KiB
clickstream  428.96 MiB
config       108.00 KiB
local         96.00 KiB


# Import Data

### Drop clickstream if Exists (Optional)

In [54]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').dropDatabase();"

{ ok: 1, dropped: 'clickstream' }


In [ ]:
!mongorestore --host localhost:27017 --db clickstream --collection clicks --drop "data\clicks.bson"
!mongoimport --host localhost:27017 --db clickstream --collection clicks.metadata --drop --type json "data\clicks.metadata.json"

In [ ]:
# 2025-06-02T19:24:52.672-0600	finished restoring clickstream.clicks (6100000 documents, 0 failures)
# 2025-06-02T19:24:52.672-0600	no indexes to restore for collection clickstream.clicks
# 2025-06-02T19:24:52.672-0600	6100000 document(s) restored successfully. 0 document(s) failed to restore.
# 2025-06-02T19:24:53.362-0600	connected to: mongodb://localhost:27017/
# 2025-06-02T19:24:53.364-0600	dropping: clickstream.clicks.metadata
# 2025-06-02T19:24:53.387-0600	1 document(s) imported successfully. 0 document(s) failed to import.

# Show Collections

In [65]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').getCollectionNames()"

[ 'clicks', 'clicks.metadata' ]


# Sample Documents

In [66]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.findOne()"

{
  _id: ObjectId('60df1029ad74d9467c91a932'),
  webClientID: 'WI100000244987',
  VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
  ProductID: 'Pr100037',
  Activity: 'click',
  device: { Browser: 'Firefox', OS: 'Windows' },
  user: { City: 'Colombo', Country: 'Sri Lanka' }
}


In [67]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.metadata.findOne()"

{
  _id: ObjectId('683e4ee50ce8cbeec786c0a5'),
  indexes: [ { v: 2, key: { _id: 1 }, name: '_id_' } ],
  uuid: 'ee6da5fe5bdf42b2bc3cecee40723af6',
  collectionName: 'clicks'
}


# Get Record Counts

In [68]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments()"

6100000


In [69]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.metadata.countDocuments()"

1


### Count of Unique Values (Field <code>device.Browser</code>)

In [71]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.distinct('device.Browser').length"

82


# Get List of Distinct Fields

In [ ]:
# limiting to 1M records (it's faster with pymongo)

In [ ]:
## list_distinct_fields.js

# (function() {
#   var fields = {};
#   db.clicks.find().limit(1000000).forEach(function(doc) {
#     Object.keys(doc).forEach(function(key) {
#       fields[key] = true;
#     });
#   });
#   printjson(Object.keys(fields));
# })();

In [2]:
!mongosh --quiet --file C:\Users\patwh\Documents\js_scripts\list_distinct_fields.js

### Get Nested Fields as Well

In [ ]:
## unique_fields_nested.js

# (() => {
#   let fields = {};

#   // recursive function to extract fields
#   function extractFields(obj, prefix = "") {

#     // iterate through keys
#     for (let key in obj) {

#       // construct full key path, e.g. parent.child
#       let fullKey = prefix + (prefix ? "." : "") + key;

#       // store field
#       fields[fullKey] = true;
#       if (obj[key]?.constructor === Object) extractFields(obj[key], fullKey);
#     }
#   }

#   // limit number of records searched
#   db.clicks.find().limit(1000000).forEach(doc => extractFields(doc));

#   // print results
#   printjson(Object.keys(fields));
# })();

In [12]:
!mongosh --quiet --file js_files\unique_fields_nested.js

[
  '_id',
  'webClientID',
  'VisitDateTime',
  'ProductID',
  'Activity',
  'device',
  'device.Browser',
  'device.OS',
  'user',
  'user.City',
  'user.Country',
  'user.UserID'
]


# Get Number of Distinct Values by Field

In [ ]:
## unique_value_counts_hardcoded.js

# (function() {
#   const collection = db.clicks;

#   const fields = [
#     '_id',
#     'webClientID',
#     'VisitDateTime',
#     'ProductID',
#     'Activity',
#     'device',
#     'device.Browser',
#     'device.OS',
#     'user',
#     'user.City',
#     'user.Country',
#     'user.UserID'
#   ];

#   // array to store fields and counts
#   const results = [];

#   // iterate through fields
#   fields.forEach(field => {

#     const pipeline = 
#       // group documents by the specified field
#       { $group: { _id: `$${field}` } },

#       // count number of documents in group 
#       { $group: { _id: null, count: { $sum: 1 } } }
#     ];

#     // run collection through pipeline, convert the result to array
#     const result = collection.aggregate(pipeline).toArray();

#     // if doc exists, extract count
#     const count = result.length > 0 ? result[0].count : 0;
#     results.push({ field: field, count: count });
#   });

#   // sort results descending by count
#   results.sort((a, b) => b.count - a.count);

#   // print sorted results
#   results.forEach(({ field, count }) => {
#     print(`${field}: ${count} unique values`);
#   });
# })();

In [14]:
!mongosh --quiet --file js_files\unique_value_counts_hardcoded_fields.js

_id: 6100000 unique values
VisitDateTime: 6089023 unique values
webClientID: 1091455 unique values
user: 72162 unique values
user.UserID: 34051 unique values
user.City: 26260 unique values
ProductID: 10938 unique values
user.Country: 222 unique values
device: 151 unique values
device.Browser: 82 unique values
device.OS: 18 unique values
Activity: 2 unique values


### Dynamic Version (First Finds Fields, then Distinct Value Counts)

In [ ]:
## count_unique_values_dynamic.js

# js_code = """
# (() => {
#   db = db.getSiblingDB('clickstream');
  
#   // empty object
#   let fields = {};

#   // extract fields from nested objects
#   function extractFields(obj, prefix = '') {
  
#     // iterate through keys
#     Object.keys(obj).forEach(key => {
    
#       // construct full key path, e.g. parent.child
#       let fullKey = prefix + (prefix ? '.' : '') + key;
      
#       // mark field as present
#       fields[fullKey] = 1;
      
#       // recurse into nested objects if valid
#       if (obj[key]?.constructor === Object) extractFields(obj[key], fullKey);
#     });
#   }

#   // iterate through documents in collection
#   db.clicks.find().forEach(extractFields);

#   // map fields to their unique value counts
#   let results = Object.keys(fields).map(field => ({
  
#     // store field name
#     field,
    
#     // count unique values
#     count: db.clicks.aggregate([
    
#       // group by field
#       { $group: { _id: `$${field}` } },
      
#       // count unique groups
#       { $group: { _id: null, count: { $sum: 1 } } }
      
#     // get first result's count or 0 if empty
#     ]).toArray()[0]?.count || 0
    
#   // sort by count in descending order
#   })).sort((a, b) => b.count - a.count);

#   // print ranked field counts
#   results.forEach((r, i) => print(`${i + 1}. ${r.field}: ${r.count}`));
# })();
# """

# Get List of Unique Values for a Field

In [17]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.distinct('device.Browser');"

[
  'AdsBot-Google',
  'AhrefsBot',
  'Amazon Silk',
  'Android',
  'AppEngine-Google',
  'Apple Mail',
  'BingPreview',
  'BlackBerry WebKit',
  'Chrome',
  'Chrome Mobile',
  'Chrome Mobile WebView',
  'Chrome Mobile iOS',
  'Chromium',
  'Coc Coc',
  'Coveobot',
  'Crosswalk',
  'Dragon',
  'DuckDuckBot',
  'Edge',
  'Edge Mobile',
  'Electron',
  'Epiphany',
  'Facebook',
  'FacebookBot',
  'Firefox',
  'Firefox Mobile',
  'Firefox iOS',
  'HbbTV',
  'HeadlessChrome',
  'HubSpot Crawler',
  'IE',
  'IE Mobile',
  'Iceweasel',
  'Iron',
  'JobBot',
  'Jooblebot',
  'K-Meleon',
  'Kindle',
  'Konqueror',
  'Magus Bot',
  'Mail.ru Chromium Browser',
  'Maxthon',
  'Mobile Safari',
  'Mobile Safari UI/WKWebView',
  'MobileIron',
  'NetFront',
  'Netscape',
  'Opera',
  'Opera Coast',
  'Opera Mini',
  'Opera Mobile',
  'Other',
  'PagePeeker',
  'Pale Moon',
  'PetalBot',
  'PhantomJS',
  'Pinterest',
  'Puffin',
  'Python Requests',
  'QQ Browser',
  'QQ Browser Mobile',
  'Radius Com

# Get Count of Unique Values for a Field

In [ ]:
## count_unique_values_for_field.js

# (function() {
#   // get clicks collection
#   const collection = db.getSiblingDB('clickstream').clicks;
  
#   // define aggregation pipeline
#   const pipeline = [
  
#     // group by Browser, count occurrences
#     { $group: { _id: "$device.Browser", count: { $sum: 1 } } },
    
#     // sort by count descending
#     { $sort: { count: -1 } }
#   ];
  
#   // run pipeline, get results
#   const result = collection.aggregate(pipeline).toArray();
  
#   // print each browser and count
#   result.forEach(doc => {
#     print(`${doc._id}: ${doc.count}`);
#   });
# })();

In [7]:
!mongosh --quiet clickstream C:/Users/patwh/Downloads/js_commands/unique_value_counts.js

Chrome: 4360498
Chrome Mobile: 788991
Firefox: 388766
Safari: 204343
Mobile Safari: 93241
HeadlessChrome: 76595
Opera: 62522
Chrome Mobile iOS: 28031
Chrome Mobile WebView: 25381
Samsung Internet: 19804
Firefox Mobile: 7040
Yandex Browser: 6265
Facebook: 5874
Opera Mobile: 5509
Edge: 5180
Coc Coc: 4506
IE: 3381
Apple Mail: 3038
Mobile Safari UI/WKWebView: 3038
UC Browser: 2142
Firefox iOS: 884
Chromium: 629
QQ Browser: 498
Jooblebot: 466
PhantomJS: 440
Amazon Silk: 376
AdsBot-Google: 312
Pinterest: 288
AhrefsBot: 229
moatbot: 216
JobBot: 208
Sogou Explorer: 200
Iron: 156
Maxthon: 139
Dragon: 105
Pale Moon: 95
PetalBot: 82
Other: 58
YandexBot: 56
Vivaldi: 43
YandexAccessibilityBot: 41
BingPreview: 33
Crosswalk: 27
Thunderbird: 23
AppEngine-Google: 22
IE Mobile: 22
Android: 19
Edge Mobile: 17
Puffin: 16
QQ Browser Mobile: 15
Epiphany: 15
Electron: 13
Opera Mini: 12
FacebookBot: 12
Yeti: 9
WordPress: 9
Coveobot: 8
PagePeeker: 6
Sleipnir: 6
Mail.ru Chromium Browser: 5
SeaMonkey: 5
HbbTV: 4

# CRUD Operations

## Delete and Create

### Identify the Last Record

In [41]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(1)"

[
  {
    _id: ObjectId('60df129dad74d9467ceebd51'),
    webClientID: 'WI100000118333',
    VisitDateTime: ISODate('2018-05-26T11:51:44.263Z'),
    ProductID: 'Pr101251',
    Activity: 'click',
    device: { Browser: 'Chrome', OS: 'Windows' },
    user: { City: 'Vijayawada', Country: 'India' }
  }
]


### Write to File

In [42]:
!mongosh --quiet --eval "print(EJSON.stringify([db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(1).toArray()[0]], {relaxed: true}))" > js_files\output.json

### Delete the Last Record

In [43]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.deleteOne({_id: db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(1).toArray()[0]._id})"

{ acknowledged: true, deletedCount: 1 }


### Insert it Back In

In [44]:
!mongoimport --db clickstream --collection clicks --file js_files\output.json --jsonArray

2025-06-02T20:42:33.812-0600	connected to: mongodb://localhost/
2025-06-02T20:42:33.819-0600	1 document(s) imported successfully. 0 document(s) failed to import.


### Identify the Last 5 Records

In [45]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(5)"

[
  {
    _id: ObjectId('60df129dad74d9467ceebd51'),
    webClientID: 'WI100000118333',
    VisitDateTime: ISODate('2018-05-26T11:51:44.263Z'),
    ProductID: 'Pr101251',
    Activity: 'click',
    device: { Browser: 'Chrome', OS: 'Windows' },
    user: { City: 'Vijayawada', Country: 'India' }
  },
  {
    _id: ObjectId('60df129dad74d9467ceebd50'),
    webClientID: 'WI1000002001',
    VisitDateTime: ISODate('2018-05-08T14:16:28.291Z'),
    ProductID: 'Pr100004',
    Activity: 'pageload',
    device: { Browser: 'HeadlessChrome', OS: 'Linux' },
    user: { City: 'Mountain View', Country: 'United States' }
  },
  {
    _id: ObjectId('60df129dad74d9467ceebd4f'),
    webClientID: 'WI100000346076',
    VisitDateTime: ISODate('2018-05-18T18:44:37.379Z'),
    ProductID: 'Pr100558',
    Activity: 'pageload',
    device: { Browser: 'Chrome Mobile', OS: 'Android' },
    user: { Country: 'India' }
  },
  {
    _id: ObjectId('60df129dad74d9467ceebd4e'),
    webClientID: 'WI100000152625',
    VisitD

### Write to File

In [49]:
!mongosh --quiet --eval "print(EJSON.stringify(db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(5).toArray(), {relaxed: true}))" > js_files\output.json

### Delete Them

In [52]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.deleteMany({_id: {$in: db.getSiblingDB('clickstream').clicks.find().sort({_id: -1}).limit(5).toArray().map(doc => doc._id)}})"

{ acknowledged: true, deletedCount: 5 }


### Insert Them Back In

In [53]:
!mongoimport --db clickstream --collection clicks --file js_files\output.json --jsonArray

2025-06-02T20:47:39.954-0600	connected to: mongodb://localhost/
2025-06-02T20:47:39.960-0600	5 document(s) imported successfully. 0 document(s) failed to import.


## Read

### Filter to <code>_id</code> Equal to <code>60df129dad74d9467ceebd51</code>

In [58]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.findOne({ _id: ObjectId('60df129dad74d9467ceebd51') })"

{
  _id: ObjectId('60df129dad74d9467ceebd51'),
  webClientID: 'WI100000118333',
  VisitDateTime: ISODate('2018-05-26T11:51:44.263Z'),
  ProductID: 'Pr101251',
  Activity: 'click',
  device: { Browser: 'Chrome', OS: 'Windows' },
  user: { City: 'Vijayawada', Country: 'India' }
}


### Find First Record Where <code>device.Browser</code> is not Firefox

In [60]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.findOne({ 'device.Browser': 'Firefox' })"

{
  _id: ObjectId('60df1029ad74d9467c91a932'),
  webClientID: 'WI100000244987',
  VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
  ProductID: 'Pr100037',
  Activity: 'click',
  device: { Browser: 'Firefox', OS: 'Windows' },
  user: { City: 'Colombo', Country: 'Sri Lanka' }
}


### Find First 2 Records Where <code>device.Browser</code> is not Firefox

In [61]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.find({ 'device.Browser': { $ne: 'Firefox' } }).limit(2).toArray()"

[
  {
    _id: ObjectId('60df1029ad74d9467c91a933'),
    webClientID: 'WI10000061461',
    VisitDateTime: ISODate('2018-05-25T05:06:03.700Z'),
    ProductID: 'Pr100872',
    Activity: 'pageload',
    device: { Browser: 'Chrome Mobile', OS: 'Android' },
    user: {}
  },
  {
    _id: ObjectId('60df1029ad74d9467c91a934'),
    webClientID: 'WI10000075748',
    VisitDateTime: ISODate('2018-05-17T11:51:09.265Z'),
    ProductID: 'Pr100457',
    Activity: 'click',
    device: { Browser: 'Chrome', OS: 'Linux' },
    user: { City: 'Ottawa', Country: 'Canada' }
  }
]


### Find First 2 Records Where <code>VisitDateTime</code> > 5/20/2018

In [62]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.find({ VisitDateTime: { $gt: ISODate('2018-05-20') } }).limit(2).toArray()"

[
  {
    _id: ObjectId('60df1029ad74d9467c91a932'),
    webClientID: 'WI100000244987',
    VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
    ProductID: 'Pr100037',
    Activity: 'click',
    device: { Browser: 'Firefox', OS: 'Windows' },
    user: { City: 'Colombo', Country: 'Sri Lanka' }
  },
  {
    _id: ObjectId('60df1029ad74d9467c91a933'),
    webClientID: 'WI10000061461',
    VisitDateTime: ISODate('2018-05-25T05:06:03.700Z'),
    ProductID: 'Pr100872',
    Activity: 'pageload',
    device: { Browser: 'Chrome Mobile', OS: 'Android' },
    user: {}
  }
]


### Get the Minimum and Maximum <code>VisitDateTime</code>

In [63]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.aggregate([{ $group: { _id: null, minVisitDateTime: { $min: '$VisitDateTime' }, maxVisitDateTime: { $max: '$VisitDateTime' } } }]).toArray()"

[
  {
    _id: null,
    minVisitDateTime: ISODate('2018-05-07T00:00:01.190Z'),
    maxVisitDateTime: ISODate('2018-05-27T23:59:59.576Z')
  }
]


Using a <code>.js</code> file instead:

In [ ]:
## get_min_and_max_date.js

# use('clickstream');
# db.clicks.aggregate([
#   {
#     $group: {
#       _id: null,
#       minVisitDateTime: { $min: "$VisitDateTime" },
#       maxVisitDateTime: { $max: "$VisitDateTime" }
#     }
#   },
#   {
#     $project: {
#       _id: 0,
#       minVisitDateTime: { $dateToString: { format: "%Y-%m-%d", date: "$minVisitDateTime" } },
#       maxVisitDateTime: { $dateToString: { format: "%Y-%m-%d", date: "$maxVisitDateTime" } }
#     }
#   }
# ]).forEach(printjson);

In [66]:
!mongosh --quiet --file js_files\get_min_and_max_date.js

{
  minVisitDateTime: '2018-05-07',
  maxVisitDateTime: '2018-05-27'
}


### Get Count of Records Where <code>VisitDateTime</code> is Greater Than 5/20/2018

In [67]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ VisitDateTime: { $gt: ISODate('2018-05-20') } })"

2453050


### Get Count of Records Where <code>user.Country</code> is <code>India</code> or <code>United States</code>

#### Using <code>$or</code>

In [68]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ $or: [{ 'user.Country': 'India' }, { 'user.Country': 'United States' }] })"

3497232


#### Using <code>$in</code>

In [69]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ 'user.Country': { $in: ['India', 'United States'] } })"

3497232


### Get Count of Records Where <code>user.Country</code> is Neither <code>India</code> Nor <code>United States</code>

#### Using <code>$and</code>

In [71]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ $and: [{ 'user.Country': { $ne: 'India' } }, { 'user.Country': { $ne: 'United States' } }] })"

2602768


#### Using `$not` and `$in`

In [72]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ 'user.Country': { $not: { $in: ['India', 'United States'] } } })"

2602768


#### Using <code>$nin</code>

In [73]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ 'user.Country': { $nin: ['India', 'United States'] } })"

2602768


### Get Count of Records with <code>user.UserID</code>

In [74]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.countDocuments({ 'user.UserID': { $exists: true, $ne: null } })"

602293


## Update

### Update <code>device.Browser</code> for Record <code>60df129dad74d9467ceebd51</code> to <code>Firefox</code>

In [75]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.updateOne({ _id: ObjectId('60df129dad74d9467ceebd51') }, { $set: { 'device.Browser': 'Firefox' } })"

{
  acknowledged: true,
  insertedId: null,
  matchedCount: 1,
  modifiedCount: 1,
  upsertedCount: 0
}


Set it back to original state for accuracy:

In [76]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.updateOne({ _id: ObjectId('60df129dad74d9467ceebd51') }, { $set: { 'device.Browser': 'Chrome' } })"

{
  acknowledged: true,
  insertedId: null,
  matchedCount: 1,
  modifiedCount: 1,
  upsertedCount: 0
}


### Update <code>device.Browser</code> Records to be <code>Firefox</code> if Set to <code>Firefox iOS</code>

In [ ]:
# !mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.updateMany({ 'device.Browser': 'Firefox iOS' }, { $set: { 'device.Browser': 'Firefox' } })"

## Create New Field

### Add Field Called <code>NewField</code> to First 1000 Records, Set Value to <code>Default</code>

In [81]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.find().limit(1000).forEach(doc => { db.getSiblingDB('clickstream').clicks.updateOne({ _id: doc._id }, { $set: { NewField: 'Default' } }); })"

From file instead:

In [ ]:
## create_new_field.js

# use('clickstream');
# db.clicks.find().limit(1000).forEach(doc => {
#   db.clicks.updateOne(
#     { _id: doc._id },
#     { $set: { NewField: "Default" } }
#   );
# });

In [83]:
!mongosh --quiet --file js_files\create_new_field.js

### Remove the Added Field

In [82]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.updateMany({ NewField: { $exists: true } }, { $unset: { NewField: '' } })"

{
  acknowledged: true,
  insertedId: null,
  matchedCount: 1000,
  modifiedCount: 1000,
  upsertedCount: 0
}


# Indexes

## View Indexes

In [85]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.getIndexes()"

[ { v: 2, key: { _id: 1 }, name: '_id_' } ]


In [87]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.metadata.findOne();"

{
  _id: ObjectId('683e62e88926dbbe2fc44be9'),
  indexes: [ { v: 2, key: { _id: 1 }, name: '_id_' } ],
  uuid: 'ee6da5fe5bdf42b2bc3cecee40723af6',
  collectionName: 'clicks'
}


## Create Indexes

In [89]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.createIndex({ 'device.OS': 1 })"

device.OS_1


In [90]:
!mongosh --quiet --eval "db.getSiblingDB('clickstream').clicks.createIndex({ 'device.Browser': 1 })"

device.Browser_1
